Lets continue with a more advanced example - trajectory optimization for 3 DOF helicopter. The cost that will be minimized will be the energy of control input on interval (0, 10).

$$\min_{x(t), \tilde{f}(t)} \int_0^{10} \tilde{f_1}^2(t) + \tilde{f_2}^2(t)\mathrm{d}t$$

where $f_i(t)$ is a thrust force produced by $i$-th motor.

The 3 DOF helicopter dynamics can be written as

$$\ddot{\lambda} = -c_{\lambda}\dot{\lambda} - \tilde{b}_{\lambda} \cos \varepsilon \sin \theta \tilde{f_s}$$
$$\ddot{\varepsilon} = a_{\varepsilon_1}\sin\varepsilon - a_{\varepsilon_2}\sin\varepsilon\cos\theta - c_{\varepsilon}\dot{\varepsilon} + \tilde{b}_{\varepsilon}\cos\theta\tilde{f_s}$$
$$\ddot{\theta} = -a_{\theta}\cos{\varepsilon}\sin\theta - c_{\theta}\dot{\theta} + b_{\theta}\tilde{f_d}$$
$$\tilde{f_s} = \tilde{f_1} + \tilde{f_2}$$
$$\tilde{f_d} = \tilde{f_1} - \tilde{f_2}$$

with following constraints

$$-90 \ \mathrm{deg} \le \theta \le 90 \ \mathrm{deg}$$
$$0 \ \mathrm{deg} \le \varepsilon \le 63.5 \ \mathrm{deg}$$
$$-360 \ \mathrm{deg} \le \lambda \le 360 \ \mathrm{deg}$$
$$0 \le \tilde{f_s} \le 1$$
$$0 \le \tilde{f_1} \le 1$$
$$0 \le \tilde{f_2} \le 1$$


Lets say we want to bring the helicopter from position ($\lambda = 0$, $\varepsilon = 30$, $\theta = 0 $) to a position ($\lambda = 90$, $\varepsilon = 30$, $\theta = 0$) - i.e. turn the travel angle by 90 degrees with no change in pitch and elevation - in the time interval with minimum energy.

The parameters were obtained by paper from former colleagues at https://gitlab.fel.cvut.cz/aa4cc/edu-ctrl-lab-models/quanser-3dof-helicopter/-/blob/master/papers/semestral_project_report_ORR_2017-18_chirtand_obrusvit.pdf?ref_type=heads 

In [4]:
using EvoOptControl

# 3 DOF helicopter dynamics
function helicopter_dynamics(x, u, t)
    λ, ε, θ, dλ, dε, dθ = x
    f₁, f₂ = u
    dx = [dλ;
          dε;
          dθ;
          -0.0001 * dλ - 0.1812 * cos(ε) * sin(θ) * (f₁ + f₂);
          0.8978 * sin(ε) - 0.2621 * sin(ε) * cos(θ) - 0.0422 * dε + 0.2423 * cos(θ) * (f₁ + f₂);
          -0.0299 * cos(ε) * sin(θ) - 0.0372 * dθ + 2.3261 * (f₁ - f₂)
        ]
    return dx
end

# Running cost (term in the integral part of the cost)
running_cost(x, u, t) = u[1]^2 + u[2]^2

# Terminal cost
terminal_cost(x, t) = 0

# Time span on which is OCP solved
tspan = (0.0, 10.0)

# Boundary conditions
x0 = [0.0; 30.0; 0.0; 0.0; 0.0; 0.0]
xf = [90.0; 30.0; 0.0; 0.0; 0.0; 0.0]

# Utility variables
state_dim = 6 # dimension of state
control_dim = 2 # dimension of control

# State & control constraints
state_constraints = [StateConstraint(identity, [-360.0, 0.0, -90.0, -Inf, -Inf, -Inf], [360.0, 63.5, 90.0, Inf, Inf, Inf])]
control_constraints = [ControlConstraint(identity, [0.0, 0.0], [1.0, 1.0]), ControlConstraint(sum, [0.0], [1.0])]

2-element Vector{ControlConstraint}:
 ControlConstraint(identity, [0.0, 0.0], [1.0, 1.0])
 ControlConstraint(sum, [0.0], [1.0])

Lets now define the direct collocation method and create 3 instances of `TrapezoidalCollocation` with different `N`.

In [5]:
N_small = 10
N_medium = 50
N_high = 200

# Trapezoidal method
method2_small = TrapezoidalCollocation(N_small)
method2_medium = TrapezoidalCollocation(N_medium)
method2_high = TrapezoidalCollocation(N_high)

TrapezoidalCollocation(200)

Now it is time do define the OCP.

In [6]:
prob2_small = OCProblem(running_cost, terminal_cost, tspan, helicopter_dynamics, x0, xf, state_dim, control_dim, method2_small, state_constraints, control_constraints)
prob2_medium = OCProblem(running_cost, terminal_cost, tspan, helicopter_dynamics, x0, xf, state_dim, control_dim, method2_medium, state_constraints, control_constraints)
prob2_high = OCProblem(running_cost, terminal_cost, tspan, helicopter_dynamics, x0, xf, state_dim, control_dim, method2_high, state_constraints, control_constraints)

problem_dict = Dict(
    ("Trapezoidal", "10") => prob2_small,
    ("Trapezoidal", "50") => prob2_medium,
    ("Trapezoidal", "200") => prob2_high
)

OCProblem{TrapezoidalCollocation}(running_cost, terminal_cost, (0.0, 10.0), helicopter_dynamics, [0.0, 30.0, 0.0, 0.0, 0.0, 0.0], [90.0, 30.0, 0.0, 0.0, 0.0, 0.0], 6, 2, TrapezoidalCollocation(200), StateConstraint[StateConstraint(identity, [-360.0, 0.0, -90.0, -Inf, -Inf, -Inf], [360.0, 63.5, 90.0, Inf, Inf, Inf])], ControlConstraint[ControlConstraint(identity, [0.0, 0.0], [1.0, 1.0]), ControlConstraint(sum, [0.0], [1.0])])

Lets now discuss, which EA will be used. I chose 2 EAs, the first one being CMA-ES with parameters $\mu = 100$ and $\lambda = 200$. The second EA will be $(\mu/\rho , \lambda)$ ES with $\mu = 100$, $\rho = 40$ and $\lambda=200$ both with population size of 300 each ran for 10 trials with 150 iterations for each run.

Crossover operator for ES is going to be averaging, mutation is going to be isotropic gaussian (one $\sigma$ for population)

In [7]:
using Evolutionary

N_runs = 10
population_size = 300
iterations = 150

# CMA-ES
ea_method1 = CMAES(mu = 100, lambda = 200)

# (mu/rho + lambda) ES
ea_method2_small = ES(initStrategy = IsotropicStrategy(N_small*(state_dim + control_dim)), srecombination = average, recombination = average, mutation = gaussian,
                   smutation = gaussian, mu = 100, rho = 40, lambda = 200, selection = :comma)
    
ea_method2_medium = ES(initStrategy = IsotropicStrategy(N_medium*(state_dim + control_dim)), srecombination = average, recombination = average, mutation = gaussian,
                    smutation = gaussian, mu = 100, rho = 40, lambda = 200, selection = :comma)
        
ea_method2_high = ES(initStrategy = IsotropicStrategy(N_high*(state_dim + control_dim)), srecombination = average, recombination = average, mutation = gaussian,
                  smutation = gaussian, mu = 100, rho = 40, lambda = 200, selection = :comma)


(100/40,200)-ES

Lets prepare storage for the optimization and lets optimize!
I'd highly suggest to use already-made figures as the optimization took a lot of time (it ran over night).

In [ ]:
# Storage for all optimization data
optimization_results = Dict{String, Dict{String, Dict{String, Vector{OCPSolution}}}}()

trapezoidal_small_cmaes = Vector{OCPSolution}(undef, N_runs)
trapezoidal_small_es = Vector{OCPSolution}(undef, N_runs)
trapezoidal_medium_cmaes = Vector{OCPSolution}(undef, N_runs)
trapezoidal_medium_es = Vector{OCPSolution}(undef, N_runs)
trapezoidal_high_cmaes = Vector{OCPSolution}(undef, N_runs)
trapezoidal_high_es = Vector{OCPSolution}(undef, N_runs)

# Main loop
for i = 1:N_runs

    sol_tr_small_cmaes = solve_ocp(prob2_small, ea_method1, population_size, iterations)
    sol_tr_small_es = solve_ocp(prob2_small, ea_method2_small, population_size, iterations)
    sol_tr_medium_cmaes = solve_ocp(prob2_medium, ea_method1, population_size, iterations)
    sol_tr_medium_es = solve_ocp(prob2_medium, ea_method2_medium, population_size, iterations)
    sol_tr_high_cmaes = solve_ocp(prob2_high, ea_method1, population_size, iterations)
    sol_tr_high_es = solve_ocp(prob2_high, ea_method2_high, population_size, iterations)

    trapezoidal_small_cmaes[i] = create_OCPSolution(prob2_small, sol_tr_small_cmaes)
    trapezoidal_small_es[i] = create_OCPSolution(prob2_small, sol_tr_small_es)
    trapezoidal_medium_cmaes[i] = create_OCPSolution(prob2_medium, sol_tr_medium_cmaes)
    trapezoidal_medium_es[i] = create_OCPSolution(prob2_medium, sol_tr_medium_es)
    trapezoidal_high_cmaes[i] = create_OCPSolution(prob2_high, sol_tr_high_cmaes)
    trapezoidal_high_es[i] = create_OCPSolution(prob2_high, sol_tr_high_es)
end

# Plots
using CairoMakie
optimization_results["Trapezoidal"] = Dict("CMAES" => Dict("10" => trapezoidal_small_cmaes, "50" => trapezoidal_medium_cmaes, "200"=>trapezoidal_high_cmaes), "ES" => Dict("10" => trapezoidal_small_es, "50" => trapezoidal_medium_es, "200" => trapezoidal_high_es))
set_theme!(theme_latexfonts(), fontsize=12)
for (method, ea_results) in optimization_results
    for (ea_type, collocation_result) in ea_results
        for (N, results) in collocation_result
            # Extracting best solution from all runs
            best_sol = minimum(results)
            prob_instance = problem_dict[(method, N)]
            # Solving ODE for pendulum given the controls found by EA
            diffeq_sol = simulate_system(helicopter_dynamics, prob_instance, best_sol)
            fig = plot_results(prob_instance, results, diffeq_sol, "Results for " * method * " with " * ea_type * " (N = " * N * ")")
            save("Results_" * method * "_" * ea_type * "_N_" * N * ".pdf", fig)
        end
    end
end

# Conclusion & results

I run a evolutionary algorithms on transcripted OCP. Local search did not do much because it converged into a local minima when controls are zero and states as well (the cost is minimized - energy is 0, penalization of initial and final state are only computed), which is not desireable.

CMA-ES and ES did find a solution (but infeasible). The constraints on boundaries were satisfied, but constraints on dynamics (state is solution to the differential equation) was not -> high value in a cost function due to penalty. What might help is to penalize the dynamics more then initial and final state constraints or maybe to use anisotropic gaussian mutation or even use maybe other kind of crossover.

It is clear that solution did not satisfy the dynamics constraint (looking at angle $\lambda$ in the figure that is in presentation - huge spike leading towards final state but the control is equal relative to the control before this occured) even from the simulation itself. I tried different ODE solvers (Tsit5 - equivalent to RK45 and Rosenbrock23 - for stiff ODEs) without significant differences.

The scaling with discretization intervals N is not negligible, about 20-50 (from my experiments) the optimization did not take too long, but for higher N it took more time: number of variables is N*(nx + nu) which is for pendulum 200 * (2 + 1) = 600 but for the helicopter it is 200 * (6 + 2) = 1600. For the 200 subintervals, it took about 7-10 sec to perform one iteration.

For the discretization methods, I did not notice significant improvemt in cost when comparing backward Euler and trapezoidal (even though trapezoidal method is more precise).

What I am sad about and what is a pity, is that I was not able to make Acados up and running. Acados serves as a tool for formulating OCPs in Matlab or Python interface and then compiling the code into a C and then it is run with solver of your choice. This should serve as a 'ground truth' or just check that the solver was not able to find feasible solution. Despite that it should clearly solve the pendulum example (since there is a pendulum on a cart example), which produces optimal solution.